In [ ]:
print("Gay")


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#PARAMETRIC PROGRAMMING

n_target = 30
n_source = 10
p = 60
K = 5
Ka = 3
h = 30
lamda = 0.03
s_vector = [0,0,0,0,0,0,0,0,0,0]
T = 5  
s = len(s_vector)
CoRT_model = CoRT(alpha=lamda)
p_values = []

iteration = 1

for step in range(iteration):
    target_data, source_data = CoRT_model.gen_data(n_target, n_source, p, K, Ka, h, s_vector, s, "AR")
    similar_source_index = CoRT_model.find_similar_source(n_target, K, target_data, source_data, T=T, verbose=False)
    X_combined, y_combined = CoRT_model.prepare_CoRT_data(similar_source_index, source_data, target_data)

    model = Lasso(alpha=lamda, fit_intercept=False, tol=1e-8, max_iter=10000000)
    model.fit(X_combined, y_combined.ravel())
    beta_hat_target = model.coef_[-p:]

    active_indices = np.array([i for i, b in enumerate(beta_hat_target) if b != 0])
    if len(active_indices) == 0:
        print(f"Iteration {iter}: Lasso selected no features. Skipping.")
        continue

    j = np.random.choice(len(active_indices))
    X_target = target_data["X"]
    y_target = target_data["y"]
    X_active, X_inactive = utils.get_active_X(beta_hat_target, X_target)
    etaj, etajTy = utils.construct_test_statistic(y_target, j, X_active)

    Sigma = np.eye(n_target)
    b_global = Sigma @ etaj @ np.linalg.pinv(etaj.T @ Sigma @ etaj)
    a_global = (Sigma - b_global @ etaj.T) @ y_target
    folds = utils.split_target(T, X_target, y_target, n_target)

    z_k = -20
    z_max = 20
    Z_train_list = utils.get_Z_train(z_k, folds, source_data, a_global, b_global, lamda, K, T)
    Z_val_list = utils.get_Z_val(z_k, folds, T, K, a_global, b_global, lamda, source_data)
    L_CoRT, R_CoRT, Az = utils.get_Z_CoRT(X_combined, similar_source_index, lamda, a_global, b_global, source_data, z_k)
    offset = p * len(similar_source_index)
    Az_target_only = np.array([idx - offset for idx in Az if idx >= offset])
    z_list = [z_k]
    Az_list = [Az_target_only]
    while z_k < z_max:
        mn = z_max
        for val in Z_train_list:
            mn = min(mn, val[4])
        for val in Z_val_list:
            mn = min(mn, val[3])
        mn = min(mn, R_CoRT)
        R_final = mn
        z_k = R_final + 0.01
        Z_train_list = utils.get_Z_train(z_k, folds, source_data, a_global, b_global, lamda, K, T)
        Z_val_list = utils.get_Z_val(z_k, folds, T, K, a_global, b_global, lamda, source_data)
        L_CoRT, R_CoRT, Az = utils.get_Z_CoRT(X_combined, similar_source_index, lamda, a_global, b_global, source_data, z_k)
        if (z_k >= z_max):
            z_list.append(z_max)
        else:
            z_list.append(z_k)
        Az_target_current = np.array([idx - offset for idx in Az if idx >= offset])
        Az_list.append(Az_target_current)

    pivot_value = utils.pivot(active_indices, Az_list, z_list, etaj, etajTy, 0, Sigma)
    p_values.append(pivot_value)
    print(f"Processing {step, pivot_value}")

# Plot results
plt.hist(p_values, density=True, bins=30, edgecolor='black')
plt.title("p-value Distribution")
plt.xlabel("p-value")
plt.ylabel('Density')
plt.tight_layout()
plt.show()
    # etaT_sigma_eta = (etaj.T @ Sigma @ etaj).item()
    # sigma_z = np.sqrt(etaT_sigma_eta)
    # truncated_cdf = utils.computed_truncated_cdf(L_final, R_final, etajTy, 0, sigma_z)
    # p_value = 2 * min(truncated_cdf, 1 - truncated_cdf)